@author: Carlos Bonilla | email: carlos.abel.bonilla@gmail.com 
@linkedIn: https://www.linkedin.com/in/carlosbonilla1/
"""

To-do Notes:
Function has a chain assignment warning; I still need to find how to prevent that potential error.

In [55]:
def investment_pv_table(periods,equipment_and_installation_cost,inventory_investment,operating_cash_flows,
                                       depreciation_period=0,discount_curve=0,marginal_tax_rate=0,
                                       equipment_salvage_value= 0, day_count = 1, interpolate_discount_curve = False):
    #day_count is meant to proxy the length of the "period" the cash flow is received in 
    # If interpolate_discount_curve is true then will use provided discount rates to interpolate a log curve
    
    #Package dependencies
    import pandas as pd

    #Create a dataframe to hold and calclate the relevant data/factors
    df = pd.DataFrame(list(range(periods+1)), columns = ["Period"]) 
    
    df['Net_investment_outlay'] = 0
    df['Net_investment_outlay'].iloc[0] = -inventory_investment-equipment_and_installation_cost
    
    df['Depreciation_Expense'] = equipment_and_installation_cost/depreciation_period
    df['Interest_Expense'] = (operating_cash_flows-df['Depreciation_Expense'])*marginal_tax_rate
    df['After_Tax_Operating_Cash_Flow'] = operating_cash_flows-df['Interest_Expense']#+df['Depreciation_Expense']
    
    df['After_Tax_Operating_Cash_Flow'].iloc[0]=0
    df['Depreciation_Expense'].iloc[0]=0
    
    df['Project_Salvage_Cash_Flow'] = 0
    df['Project_Salvage_Cash_Flow'].iloc[depreciation_period] = equipment_salvage_value+inventory_investment-(marginal_tax_rate*equipment_salvage_value) 
    df['Project_Cash_Flow'] = df['Net_investment_outlay']+df['After_Tax_Operating_Cash_Flow']+df['Project_Salvage_Cash_Flow']
    
    if interpolate_discount_curve == True:
        import numpy as np
        df['Discount_Rate'] = np.geomspace(discount_curve[0], discount_curve[1], num=len(cash_flows))
    else: df['Discount_Rate'] = discount_curve

    #Calculate discount factors
    df['Discount_Factor'] = 1/((1+df['Discount_Rate'])**(df['Period']*day_count))

    #Can loop through the periods and calculate each PV or multiple the cash flow by the discount factor
    df['Project_Cash_Flow_PV']= df['Project_Cash_Flow'] * df['Discount_Factor']
    
    #Economic income is the cash flow plus the change in value, or economic income is the cash flow minus the economic depreciation
    #df['Economic_Income'] = df['PV'].iloc[::-1]-df['Project_Cash_Flow']

    #Return a pandas dataframe 
    return df

In [56]:
investment_pv_table(periods=2,equipment_and_installation_cost=100,inventory_investment=0,operating_cash_flows=50+50,
                    depreciation_period=2,discount_curve=.12,marginal_tax_rate=.3,equipment_salvage_value=0)

C:\Users\Carlos Bonilla\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Period,Net_investment_outlay,Depreciation_Expense,Interest_Expense,After_Tax_Operating_Cash_Flow,Project_Salvage_Cash_Flow,Project_Cash_Flow,Discount_Rate,Discount_Factor,Project_Cash_Flow_PV
0,0,-100,0.0,15.0,0.0,0.0,-100.0,0.12,1.000000,-100.000000
1,1,0,50.0,15.0,85.0,0.0,85.0,0.12,0.892857,75.892857
2,2,0,50.0,15.0,85.0,0.0,85.0,0.12,0.797194,67.761480


In [58]:
npv = investment_pv_table(periods=2,equipment_and_installation_cost=100,inventory_investment=0,operating_cash_flows=50+50,
                    depreciation_period=2,discount_curve=.12,marginal_tax_rate=.3,equipment_salvage_value=0)['Project_Cash_Flow_PV'].sum()
npv

C:\Users\Carlos Bonilla\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


43.65433673469387